In [3]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import plotly.express as px

import math
from datetime import datetime
import calendar
import pytz

import warnings

warnings.filterwarnings("ignore")

from typing import List

from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split

import keras

import model_prep

In [4]:
building_name="ESB"
tower_number=1
season="summer"
esb1_features = ['ESB_Tower_1 enteringWaterTemp', 'ESB_Tower_1 outdoorAirDryBulb', 'ESB_Tower_1 outdoorAirWetBulb', 'ESB_Tower_1 vfdPercent', 'ESB_Tower_1 fanA_vfdPower', 'ESB_Tower_1 fanB_vfdPower', 'ESB_Tower_1 dayOfWeek', 'ESB_Tower_1 hourOfDay']
esb1_target = 'ESB_Tower_1 leavingWaterTemp'
plot_history = False
test_split_percentage = 0.2
use_derivatives = True

features = esb1_features
target = esb1_target

step_back = 6  # window size = 6*5 = 30 mins
season_map = {
    "spring": [3, 4, 5],
    "summer": [6, 7, 8],
    "fall": [9, 10, 11],
    "winter": [12, 1, 2],
}
year = 2022

In [ ]:
"""
1. Load data and do LSTM preprocessing
"""
# load data
df = pd.read_csv(
        f"../data/{building_name.lower()}/{building_name.lower()}_tower_{tower_number}_preprocessed.csv",
        index_col="time",
)
df.index = pd.to_datetime(df.index)

# only take data for one season
df = model_prep.choose_season(
    df,
    season=season,
    season_col_name=f"{building_name}_Tower_{tower_number} season",
)

# save a boolean series that specifies whether the cooling tower is on
on_condition = df[f"{building_name}_Tower_{tower_number} fanStatus"]

# select features and targets and create final dataframe that includes only relevant features and targets
df = df[features].join(df[target], on=df.index)

first_temp = df.iloc[0, df.columns.get_loc(target)]
df[target] = df[target] - first_temp

# normalize data
scaler = model_prep.NormalizationHandler()
df = scaler.normalize(df, target_col=target)

# prepare dataframe for lstm by adding timesteps
lstm_df = model_prep.create_timesteps(
    df, n_in=step_back, n_out=1, target_name=target
)

# remove cases where spring data would leak into summer data (i.e. intial timesteps)
lstm_df = model_prep.remove_irrelevant_data(
    lstm_df, on_condition, step_back
)

In [ ]:
X = lstm_df.drop(f"{target}(t)", axis=1)  # drop target column
y = lstm_df[f"{target}(t)"]  # only have target column

# split into input and outputs
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_split_percentage, shuffle=False)

In [ ]:
"""
3. Get timestepped data as a 3D vector
"""
vec_X_train = model_prep.df_to_3d(
    X_train, num_columns=len(features) + 1, step_back=step_back
)
vec_X_test = model_prep.df_to_3d(
    X_test, num_columns=len(features) + 1, step_back=step_back
)

vec_y_train = y_train.values
vec_y_test = y_test.values

In [ ]:
"""
4. Create and Train model
"""
model = keras.models.Sequential()
model.add(keras.layers.LSTM(40, input_shape=(vec_X_train.shape[1], vec_X_train.shape[2])))
model.add(keras.layers.Dense(1))
model.compile(loss="mae", optimizer="adam")

In [ ]:
history = model.fit(
    vec_X_train,
    vec_y_train,
    epochs=50,
    batch_size=72,
    validation_data=(vec_X_test, vec_y_test),
    verbose=0,
    shuffle=False,
)

In [ ]:
# make predictions
yhat = model.predict(vec_X_test)

In [ ]:
"""
5. Display results
"""
results_df = pd.DataFrame(
        {
            "actual": vec_y_test.reshape((vec_y_test.shape[0])),
            "predicted": yhat.reshape((yhat.shape[0])),
        },
        index=y_test.index,
    )
results_df = scaler.denormalize_results(results_df)
results_df["actual"] = results_df["actual"] + first_temp
results_df["predicted"] = results_df["predicted"] + first_temp

# Create a new DataFrame with the desired 5-minute interval index
new_index = pd.date_range(
    start=pytz.utc.localize(datetime(year, season_map[season][0], 1)),
    end=pytz.utc.localize(
        datetime(
            year,
            season_map[season][-1],
            calendar.monthrange(year, season_map[season][-1])[1],
        )
    ),
    freq="5min",
)
display_df = pd.DataFrame(index=new_index)
# Merge the new DataFrame with the original DataFrame
display_df = display_df.merge(
    results_df, how="left", left_index=True, right_index=True
)

mabs_error = mean_absolute_error(results_df["actual"], results_df["predicted"])
rmse = np.sqrt(mean_squared_error(results_df["actual"], results_df["predicted"]))
print("Mean Absolute Error: %.3f" % mabs_error)
print("RMSE: %.3f" % rmse)

fig = px.line(display_df, x=display_df.index, y=["actual", "predicted"])
fig.update_layout(
    title=f"{building_name} Tower {tower_number} LSTM Model Results",
    xaxis_title="time",
    yaxis_title=target,
)
fig.show()

# Transfer to Kissam

In [ ]:
building_name="ESB"
tower_number=2
season="summer"
esb2_features = ['ESB_Tower_2 enteringWaterTemp', 'ESB_Tower_2 outdoorAirDryBulb', 'ESB_Tower_2 outdoorAirWetBulb', 'ESB_Tower_2 vfdPercent', 'ESB_Tower_2 fanA_vfdPower', 'ESB_Tower_2 fanB_vfdPower', 'ESB_Tower_2 dayOfWeek', 'ESB_Tower_2 hourOfDay']
esb2_target = 'ESB_Tower_2 leavingWaterTemp'
plot_history = False
test_split_percentage = 0.2
use_derivatives = True

features = esb2_features
target = esb2_target

step_back = 6  # window size = 6*5 = 30 mins
season_map = {
    "spring": [3, 4, 5],
    "summer": [6, 7, 8],
    "fall": [9, 10, 11],
    "winter": [12, 1, 2],
}
year = 2022

In [ ]:
"""
1. Load data and do LSTM preprocessing
"""
# load data
df = pd.read_csv(
        f"../data/{building_name.lower()}/{building_name.lower()}_tower_{tower_number}_preprocessed.csv",
        index_col="time",
)
df.index = pd.to_datetime(df.index)

# only take data for one season
df = model_prep.choose_season(
    df,
    season=season,
    season_col_name=f"{building_name}_Tower_{tower_number} season",
)

# save a boolean series that specifies whether the cooling tower is on
on_condition = df[f"{building_name}_Tower_{tower_number} fanStatus"]

# select features and targets and create final dataframe that includes only relevant features and targets
df = df[features].join(df[target], on=df.index)

first_temp = df.iloc[0, df.columns.get_loc(target)]
df[target] = df[target] - first_temp

# normalize data
scaler = model_prep.NormalizationHandler()
df = scaler.normalize(df, target_col=target)

# prepare dataframe for lstm by adding timesteps
lstm_df = model_prep.create_timesteps(
    df, n_in=step_back, n_out=1, target_name=target
)

# remove cases where spring data would leak into summer data (i.e. intial timesteps)
lstm_df = model_prep.remove_irrelevant_data(
    lstm_df, on_condition, step_back
)

In [ ]:
X = lstm_df.drop(f"{target}(t)", axis=1)  # drop target column
y = lstm_df[f"{target}(t)"]  # only have target column

# split into input and outputs
X_test, y_test = X, y

In [ ]:
# create 3d vector form of data
vec_X_test = model_prep.df_to_3d(
        lstm_dtframe=X_test, num_columns=len(features) + 1, step_back=step_back
    )
vec_y_test = y_test.values

# load model
# model = keras.models.load_model(
#     f"../models_saved/esb1_summer_lstm/"
# )

In [ ]:
yhat = model.predict(vec_X_test)

In [ ]:
"""
5. Display results
"""
results_df = pd.DataFrame(
        {
            "actual": vec_y_test.reshape((vec_y_test.shape[0])),
            "predicted": yhat.reshape((yhat.shape[0])),
        },
        index=y_test.index,
    )
results_df = scaler.denormalize_results(results_df)
results_df

In [ ]:
results_df["actual"] = results_df["actual"] + first_temp
results_df["predicted"] = results_df["predicted"] + first_temp

# Create a new DataFrame with the desired 5-minute interval index
new_index = pd.date_range(
    start=pytz.utc.localize(datetime(year, season_map[season][0], 1)),
    end=pytz.utc.localize(
        datetime(
            year,
            season_map[season][-1],
            calendar.monthrange(year, season_map[season][-1])[1],
        )
    ),
    freq="5min",
)
display_df = pd.DataFrame(index=new_index)
# Merge the new DataFrame with the original DataFrame
display_df = display_df.merge(
    results_df, how="left", left_index=True, right_index=True
)

mabs_error = mean_absolute_error(results_df["actual"], results_df["predicted"])
rmse = np.sqrt(mean_squared_error(results_df["actual"], results_df["predicted"]))
print("Mean Absolute Error: %.3f" % mabs_error)
print("RMSE: %.3f" % rmse)

fig = px.line(display_df, x=display_df.index, y=["actual", "predicted"])
fig.update_layout(
    title=f"{building_name} Tower {tower_number} LSTM Model Results",
    xaxis_title="time",
    yaxis_title=target,
)
fig.show()